In [14]:
############ LIBRARIES (updated) ###############

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import csv
import pycatch22

# Regressors
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb

import warnings
from warnings import simplefilter
warnings.filterwarnings("ignore", category=RuntimeWarning)
simplefilter(action='ignore', category=FutureWarning)

# Evaluation metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

%matplotlib inline

# variavel que evita NaN nos resultados
epslon = 0.00005


def pbe(y_true, y_pred):
  if np.sum(y_true)!=0:
    return 100*(np.sum(y_pred - y_true)/np.sum(y_true))
  else:
   return 100*(np.sum(y_pred - y_true)/(np.sum(y_true)+ epslon))  

def pocid(y_true, y_pred):
  n = len(y_true)
  D = [1 if (y_pred[i] - y_pred[i-1]) * (y_true[i] - y_true[i-1]) > 0 else 0 for i in range(1, n)]
  POCID = 100 * np.sum(D) / (n-1)
  return POCID

#função para normalização
def znorm(x):
  if np.std(x)!=0:
    x_znorm = (x - np.mean(x)) / np.std(x)
  else:
    x_znorm = (x - np.mean(x)) / (np.std(x) + epslon) 
   
  return x_znorm

#função para desnormatização
def znorm_reverse(x, mean_x, std_x):
  x_denormalized = (np.array(x) * std_x) + mean_x
  return x_denormalized

def get_stats_norm(series, horizon, window):
  last_subsequence = series[-(horizon+window):-horizon].values
  last_mean = np.mean(last_subsequence)
  last_std = np.std(last_subsequence)
  return last_mean, last_std

# Em geral, considera-se um tamanho de janela capaz de capturar um ciclo dos dados
# Por exemplo, 12 observações no caso dos dados com frequência mensal
def rolling_window(series, window):
  data = []
  for i in range(len(series)-window):
    example = znorm(np.array(series[i:i+window+1]))
    data.append(example)
  df = pd.DataFrame(data)
  return df

# Para predição de vendas por UF (mensal), será considerado horizon = 12
# Para predição de vendas por município (anual), será considerado horizon = 1
def train_test_split(data, horizon):
  X = data.iloc[:,:-1] # features
  y = data.iloc[:,-1] # target

  X_train = X[:-horizon] # features train
  X_test =  X[-horizon:] # features test

  y_train = y[:-horizon] # target train
  y_test = y[-horizon:] # target test
  return X_train, X_test, y_train, y_test

def recursive_multistep_forecasting(X_test, model, horizon):
  # example é composto pelas últimas observações vistas
  # na prática, é o pbeprimeiro exemplo do conjunto de teste
  example = X_test.iloc[0].values.reshape(1,-1)

  preds = []
  for i in range(horizon):
    pred = model.predict(example)[0]
    preds.append(pred)

    # Descartar o valor da primeira posição do vetor de características
    example = example[:,1:]

    # Adicionar o valor predito na última posição do vetor de características
    example = np.append(example, pred)
    example = example.reshape(1,-1)
  return preds

#função Features PyCatch22 
def rolling_window_featureCatch22(series, window):
  data = []
  for i in range(len(series)-window):
      example = np.array(series[i:i+window])
      new_elements = pycatch22.catch22_all(example)
      data_feature = (new_elements['values'])
      data.append(data_feature)
  df = pd.DataFrame(data)
  return df


In [15]:
#### LEITURA DOS DADOS ####

def extract_estado(file_name):
    # Split the file name by underscores
    parts = file_name.split('_')
    # Extract the name between underscores
    estado = parts[1]
    return estado

def read_csv_files(folder_path):
    estados = []
    # List all files in the folder
    files = os.listdir(folder_path)
    # Iterate through each file
    for file_name in files:
        # Check if it's a CSV file
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            # Open the CSV file and read the data
            with open(file_path, 'r', newline='') as csvfile:
                reader = csv.reader(csvfile)
                # Assuming the first row contains headers
                headers = next(reader)
                # Extract estado from file name and append to estados list
                estado = extract_estado(file_name)
                estados.append(estado)
                estados.sort()
    return estados



In [16]:
############## Catch22 AUTO ##############

import pickle

horizon = 12
window = 12
features = 'Catch22'

products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
 
for product in products:
    folder_path = f'./uf/{product}/'
    # Read the CSV files and extract estado names
    estados = read_csv_files(folder_path)
    
    for estado in estados:

        # carregamento do arquivo

        df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
        series = df['m3']

        TimeStamp = df['timestamp'].tail(398)
        TimeStamp.reset_index(drop=True, inplace=True)

        Catch22 = rolling_window_featureCatch22(series, window)
        Catch22_filled = Catch22.fillna(0)

        # first_column = Catch22_filled.pop('timestamp')
        Catch22_filled.insert(0, 'timestamp', TimeStamp)

        folder_name = 'Catch22'
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        
        Catch22_filled.to_csv(f'{folder_name}/FEAT_Catch22_{product}_{window}_{estado}.csv', index=False)


KeyboardInterrupt: 

In [9]:
Catch22_filled


,timestamp,0,1,2,3,4,5,6,7,8,...,13,14,15,16,17,18,19,20,21,22
0,199101,-0.866419,-0.370693,0.559391,2.0,1.054920,-2.223682,1.000000,3.0,0.005510,...,-0.083333,0.000000,2.567907e-34,4.0,2.145842,0.0,0.0,1.963495,0.926007,1342.0115
1,199102,-0.875795,-1.046924,0.788129,2.0,0.845621,1.149659,1.000000,5.0,0.080000,...,0.166667,0.083333,2.567907e-34,4.0,1.893788,0.0,0.0,1.178097,1.300028,1439.1240
2,199103,-0.922045,-0.396486,0.738073,2.0,0.845621,1.235168,1.000000,5.0,0.080000,...,0.000000,0.666667,0.000000e+00,4.0,1.893788,0.0,0.0,1.178097,1.295246,1426.5950
3,199104,-0.822207,-0.995896,0.828035,2.0,0.595420,1.092709,0.909091,4.0,0.080000,...,-0.166667,0.666667,2.567907e-34,4.0,1.720193,0.0,0.0,1.178097,1.263712,1656.6190
4,199105,-0.769977,-0.942881,0.857548,2.0,0.475135,1.085301,0.909091,3.0,0.080000,...,-0.250000,0.500000,1.604942e-33,4.0,1.846220,0.0,0.0,1.178097,1.266665,1750.8820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,202310,0.611195,0.431738,0.489152,1.0,0.586707,1.815440,0.636364,3.0,0.038567,...,0.166667,-0.083333,0.000000e+00,2.0,1.641735,0.0,0.0,1.963495,0.946102,403.0000
394,202311,-0.398030,-0.215544,0.538752,2.0,0.752394,4.134022,0.636364,3.0,0.038567,...,0.500000,-0.250000,1.027163e-33,3.0,1.641735,0.0,0.0,1.963495,1.007126,381.0000
395,202312,-0.230143,-0.034699,0.586903,3.0,0.718414,4.480588,0.636364,4.0,0.055096,...,0.333333,0.583333,1.027163e-33,4.0,1.540306,0.0,0.0,1.570796,1.396861,505.0000
396,202401,-0.282109,-0.104139,0.519407,1.0,0.752394,4.637785,0.727273,4.0,0.055096,...,0.583333,0.416667,4.108651e-33,5.0,1.540306,0.0,0.0,1.570796,1.414228,340.0000
